In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40
path = "../data/ModelNet40"
categories = ['bottle']
tr, te = get_dataloaders(path, categories=categories)

Loading renders for bottle: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 16.44it/s]


In [ ]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features_list": [32, 64, 128, 192, 192, 256],
    "num_layers_list": 1,
    "attn_heads": (None, None, None, 8, 8)
}]
up_blocks = [
    {
        "features_list": [256, 192, 192],
        "num_layers_list": 2,
        "attn_heads": 8
    },
    {
        "features_list": [192, 128, 64, 32],
        "num_layers_list": 2,
        "attn_heads": None
    },
]
t_emb_features = 64

model = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

lr = 1e-4
model = models.DiffusionBase(model, lr=lr)

In [ ]:
import lightning as L
trainer = L.Trainer(
    max_epochs=100,
    gradient_clip_val=10.0,
)

trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)

In [ ]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model.cuda(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)

In [3]:
import models

m = models.SPVD()

lr = 1e-4
model2 = models.DiffusionBase(m, lr=lr)

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("../logs/ModelNet40/bottle2", name="test")

checkpoint_name = "-".join(categories) if categories else "all"
epochs = 40
checkpoint_callback = ModelCheckpoint(
    dirpath=f'../checkpoints/ModelNet40/{checkpoint_name}',
    filename="{epoch:02d}-{val_loss:.2f}", 
    every_n_epochs=5
)

trainer = L.Trainer(
    logger=logger,
    log_every_n_steps=5,
    max_epochs=epochs, 
    gradient_clip_val=10.0, 
    callbacks=[checkpoint_callback]
)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model=model2, train_dataloaders=tr, val_dataloaders=te)

You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/ubuntu/SPVD_Lightning/checkpoints/ModelNet40/bottle exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 32.9 M | train
------------------------------------------
32.9 M    Trainable params
0         Non-trainable params
32.9 M    Total params
131.576   Total estimated model params size (MB)
314       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/envs/spvd/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/ubuntu/SPVD_Lightning/src/datasets/modelnet40/modelnet40_loader.py", line 105, in __getitem__
    data = super().__getitem__(idx)
  File "/home/ubuntu/SPVD_Lightning/src/datasets/modelnet40/modelnet40_loader.py", line 73, in __getitem__
    renders = self.render_features[idx]
AttributeError: 'ModelNet40Sparse' object has no a

In [ ]:
from utils.schedulers import DDPMSparseSchedulerGPU

ddpm_sched = DDPMSparseSchedulerGPU(n_steps=1000, beta_min=0.0001, beta_max=0.02, pres=1e-5)

preds = ddpm_sched.sample(model2.cuda(), 16, 2048)

from utils.visualization import visualize_notebook
visualize_notebook(preds, x_offset=2.5, y_offset=2.5, point_size=0.025)